In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from utils import *
from tqdm import tqdm

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
d = 3
m = 8
t = 200
n = 50000
snr = 25
lamda = 0.1

array = ArrayModel(m, lamda)

observations, angles, perturbations_x, perturbations_y = generate_data(n, t, d, snr, array)

In [3]:
class my_model(nn.Module): 
    
    def __init__(self, m: int, d: int, device=dev):
        
        super().__init__()

        self.m = m
        self.d = d
        self.device = device

        self.bn = nn.BatchNorm1d(2*m, device=device)
        self.rnn = nn.GRU(input_size=2*m, hidden_size=2*m, num_layers=1, batch_first=True, device=device)

        self.mlp = nn.Sequential(nn.Linear(in_features=2*m, out_features=2*m*m, device=device), nn.ReLU(),
                                 nn.Linear(in_features=2*m*m, out_features=2*m*m, device=device), nn.ReLU(),
                                 nn.Linear(in_features=2*m*m, out_features=d, device=device))

    def forward(self, x):
        
        x = torch.cat((torch.real(x), torch.imag(x)), dim=-1)
        x = self.bn(x.transpose(1, 2)).transpose(1, 2)
        _, x = self.rnn(x)
        y = self.mlp(x[-1])

        return y


In [4]:
nbEpoches = 200
lr = 5e-3
wd = 1e-9
batchSize = 256

x_train, x_valid, theta_train, theta_valid, del_x_train, del_x_valid, del_y_train, del_y_valid = train_test_split(observations, angles, perturbations_x, perturbations_y, test_size=0.2)
x_train, x_test, theta_train, theta_test, del_x_train, del_x_test, del_y_train, del_y_test = train_test_split(x_train, theta_train, del_x_train, del_y_train, test_size=0.2)

train_set = DATASET(x_train, theta_train, del_x_train, del_y_train)
valid_set = DATASET(x_valid, theta_valid, del_x_valid, del_y_valid)
test_set = DATASET(x_test, theta_test, del_x_test, del_y_test)

train_loader = DataLoader(train_set, batch_size=batchSize, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batchSize, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batchSize, shuffle=False)

In [ ]:
model = my_model(m, d)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
Loss, Val = [], []
bestVal = 1000

for i in tqdm(range(nbEpoches)):
    # Train
    running_loss = 0.0
    for data in train_loader:
        X, theta_true, del_x_true, del_y_true = data[0].to(dev), data[1].to(dev), data[2].to(dev), data[3].to(dev)
        optimizer.zero_grad()
        theta_pred = model(X)
        loss = RMSPE(theta_pred, theta_true) 
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    Loss.append(running_loss/len(train_loader))

    # Validation 
    with torch.no_grad():
        running_loss = 0.0
        for data in valid_loader:
            X, theta_true, del_x_true, del_y_true = data[0].to(dev), data[1].to(dev), data[2].to(dev), data[3].to(dev)
            theta_pred = model(X)
            loss = RMSPE(theta_pred, theta_true)
            running_loss += loss.item()
        
        Val.append(running_loss/len(test_loader))

        if Val[i] < bestVal:
            bestVal = Val[i]
            torch.save(model.state_dict(), 'model_3.pt')

    print("Iteration {}: Loss = {}".format(i, Loss[-1]))

In [ ]:
model_test = my_model(m, d)
model_test.load_state_dict(torch.load('model_3.pt', weights_only=True))
with torch.no_grad():
    running_loss = 0.0
    for data in test_loader:
        X, theta_true, del_x_true, del_y_true = data[0].to(dev), data[1].to(dev), data[2].to(dev), data[3].to(dev)
        theta_pred = model_test(X)
        loss = RMSPE(theta_pred, theta_true)
        running_loss += loss.item()

Acc = running_loss/len(test_loader)

print("RMSE DoA = ", Acc)